In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [3]:
model = pickle.loads( open('dnn_model_keras', 'rb').read())
dict_dataset = pickle.loads(open('dict_dataset', 'rb').read())

train_X = dict_dataset['train_X']
test_X = dict_dataset['test_X']
train_y = dict_dataset['train_y']
test_labels = dict_dataset['test_labels']


W0907 20:53:56.091554 4604163520 deprecation_wrapper.py:119] From /Users/mcoria/anaconda3/envs/customenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0907 20:53:56.135589 4604163520 deprecation_wrapper.py:119] From /Users/mcoria/anaconda3/envs/customenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0907 20:53:56.220180 4604163520 deprecation_wrapper.py:119] From /Users/mcoria/anaconda3/envs/customenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0907 20:53:56.220973 4604163520 deprecation_wrapper.py:119] From /Users/mcoria/anaconda3/envs/customenv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecate

In [4]:
test_probs = model.predict(test_X)
test_predictions = np.argmax(test_probs, axis=1)
train_labels = train_y
train_probs=  model.predict(train_X)
train_predictions = np.argmax(train_probs, axis=1)
#print(confusion_matrix(test_labels, y_pred))





In [ ]:

from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Plot formatting
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18


def evaluate_model(test_labels, predictions, probs, train_labels, train_predictions, train_probs):
    
    baseline = {}
    
    baseline['recall'] = recall_score(test_labels, 
                                     [1 for _ in range(len(test_labels))])
    baseline['precision'] = precision_score(test_labels, 
                                      [1 for _ in range(len(test_labels))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(test_labels, predictions)
    results['precision'] = precision_score(test_labels, predictions)
    results['roc'] = roc_auc_score(test_labels, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(str(metric.capitalize())+ 'Baseline: ' + str(round(baseline[metric], 2)) + ' Test: '+ str(round(results[metric], 2)) + ' Train: ' +str(round(train_results[metric], 2)))
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_labels, [1 for _ in range(len(test_labels))])
    model_fpr, model_tpr, _ = roc_curve(test_labels, probs)

    plt.figure(figsize = (8, 8))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); 
    plt.ylabel('True Positive Rate'); 
    plt.title('ROC Curves');
    plt.show();

    
evaluate_model(test_labels, test_predictions, test_probs, train_labels, train_predictions, train_probs)

plt.savefig('roc_auc_curve_50k_2.png')


